In [1]:
#!pip install openai
#!pip install python-dotenv


In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
from langchain.chat_models import ChatOpenAI

llm_model = "gpt-3.5-turbo"

chat = ChatOpenAI(temperature=0.9, model=llm_model)


### Langchain prompt
Use langchain to ask for more complex prompt to generate detailed responce

In [5]:

template_string = """You are IT Recruitment specialist, 
looking for expert in {job_advert} 
you need to analyze following candidate {cv_general} having experience {cv_experience}
let me know following details :
cv_fit_jd : clear statement about how well the candidate's experience fits the job description, on how many skills
cv_fit_jd_score : score of the candidate's experience fit to the job description 10 is high 0 is low
cv_suggestions : suggestions for the candidate to improve their CV"""

from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)


#load job advert from file
job_advert = open("job_advert.txt", "r").read()
#load cv_general from file
cv_general = open("cv_general.txt", "r").read()
#load cv_experience from file
cv_experience = open("cv_experience.txt", "r").read()


customer_messages = prompt_template.format_messages(job_advert=job_advert, cv_general=cv_general, cv_experience=cv_experience)

service_response = chat(customer_messages)
print(service_response.content)



cv_fit_jd: The candidate's experience fits well with the job description as they have extensive experience with Python, Django, DjangoRestFramework, GitLab, and OpenShift, which are all mentioned in the job description. They also have experience with CI/CD pipelines, container orchestration, and software architecture tasks.

cv_fit_jd_score: 8

cv_suggestions: 
1. Highlight your experience with AWS Cloud Development as it is a must-have skill for the job.
2. Consider adding your experience with Terraform for Infrastructure as Code, as it is mentioned as a should-have skill.
3. Emphasize your problem-solving skills with tools such as Cloudwatch, Athena, DynamoDB, and Quicksight.
4. Include any experience with FastAPI, Pytest, Pydantic, and Poetry Python frameworks, as they are listed as should-have skills.
5. Provide more details about your experience with ERD diagrams, flow charts, and software architecture tasks to showcase your expertise in this area.


NameError: name 'ResponseSchema' is not defined

### Langchain parsers
use langchain LLM parser to store string into dictionaries

In [7]:
from langchain.output_parsers import ResponseSchema,StructuredOutputParser


In [8]:


cv_fit_jd_schema = ResponseSchema(name="cv_fit_jd",
                                  description="""Clear statement about how well the candidate's experience 
                                  fits the job description, on how many skills""")

cv_fit_jd_score_schema = ResponseSchema(name="cv_fit_jd_score",
                                        description="Score of the candidate's experience fit to the job description. 10 is high, 0 is low")

cv_suggestions_schema = ResponseSchema(name="cv_suggestions",
                                       description="Suggestions for the candidate to improve their CV")

response_schemas = [cv_fit_jd_schema, 
                    cv_fit_jd_score_schema,
                    cv_suggestions_schema]


output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"cv_fit_jd": string  // Clear statement about how well the candidate's experience 
                                  fits the job description, on how many skills
	"cv_fit_jd_score": string  // Score of the candidate's experience fit to the job description. 10 is high, 0 is low
	"cv_suggestions": string  // Suggestions for the candidate to improve their CV
}
```


In [11]:
template_string = """You are IT Recruitment specialist, 
looking for expert in {job_advert} 
you need to analyze following candidate {cv_general} having experience {cv_experience}
{format_instructions}"""

prompt_template = ChatPromptTemplate.from_template(template_string)
customer_messages= prompt_template.format_messages(job_advert=job_advert, cv_general=cv_general, cv_experience=cv_experience, format_instructions=format_instructions)

service_response = chat(customer_messages)
print(service_response.content)


KeyError: 'format_instructions'